<a href="https://colab.research.google.com/github/albertochumpitazi/Master_VIU_AlbertoChumpitazi/blob/main/03_Transformacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## <center>  <center>
## <center> Universidad Internacional de Valencia (VIU) <center>
## <center> Trabajo Fin de Master (TFM) <center>


---


# <center> **Extracción de patrones de comportamiento de consumo de energía de los hogares en una red inteligente** <center>


---



**Titulación:** Máster en Big Data y Data Science

**Alumno:** Alberto Sergio Chumpitazi Llave

**Director:** Jose Carlos González, PhD.

#4.3. Transformación

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import warnings
import pandas as pd
warnings.filterwarnings('ignore')

cTotal = ['Total Consumo']
cTemp = ['Temp']

cArtefCocin = ['Dish washer','Microwave','Kettle' ]
cArtefLavan = ['Washing Machine', 'Dryer']
cArtefConti = ['Water heater', 'Refrigerator']
cArtefIlumi = ['Lighting']
cArtefEntre = ['TV']

cArtefactos = cArtefCocin + cArtefLavan + cArtefConti + cArtefIlumi + cArtefEntre
cArtefactosTemp  = cArtefactos  + cTemp

ruta_excel = '/content/drive/MyDrive/VIU/14MBID/Data/02_Procesado.csv'
df_base = pd.read_csv(ruta_excel, sep=",")
df_base['fecha'] = pd.to_datetime(df_base['fecha'])
df_base

,fecha,hora,Consumidor,Dish washer,Washing Machine,Dryer,Water heater,TV,Microwave,Kettle,Lighting,Refrigerator,Temp,Producer,Is_producer
0,2013-08-01,0,Consumer1,0.000000,0.000000,0.0,0.000,0.000000,0.0,0.0,0.000000,0.14,24,0.0,True
1,2013-08-01,0,Consumer10,0.000000,0.000000,0.0,0.000,0.000000,0.0,0.0,0.000000,0.28,24,0.0,True
2,2013-08-01,0,Consumer11,0.000000,0.000000,0.0,1.178,0.000000,0.0,0.0,0.000000,0.00,24,0.0,True
3,2013-08-01,0,Consumer12,0.000000,0.000000,0.0,1.005,0.000000,0.0,0.0,0.000000,0.00,24,0.0,True
4,2013-08-01,0,Consumer13,0.000000,0.000000,0.0,2.382,0.000000,0.0,0.0,0.000000,0.14,24,0.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355865,2014-08-01,23,Consumer47,0.000000,0.000000,0.0,0.000,0.000000,0.0,0.0,0.000000,0.14,12,0.0,False
355866,2014-08-01,23,Consumer49,1.251331,0.000000,0.0,0.000,0.000000,0.0,0.0,0.000000,0.00,12,0.0,False
355867,2014-08-01,23,Consumer5,0.000000,1.882762,0.0,0.000,0.000000,0.0,0.0,0.000000,0.00,12,0.0,True
355868,2014-08-01,23,Consumer50,0.000000,0.000000,0.0,0.659,0.093508,0.0,0.0,0.499667,0.00,12,0.0,False


In [3]:
df_transformacion = df_base.copy()
df_transformacion.insert(1, 'Total Consumo', df_transformacion[cArtefactos].sum(axis=1))

In [4]:
rangos_horarios = {
    '1.Madrugada': (0, 6),
    '2.Mañana': (6, 12),
    '3.Tarde': (12, 18),
    '4.Noche': (18, 24)
}

rangos_estacionales = {
    'Invierno2': ((1, 1), (3, 20)),
    '2.Primavera': ((3, 21), (6, 20)),
    '3.Verano': ((6, 21), (9, 22)),
    '4.Otoño': ((9, 23), (12, 20)),
    '1.Invierno': ((12, 21), (12, 31))
}

dias_semana = {
    0: 'Lunes',
    1: 'Martes',
    2: 'Miércoles',
    3: 'Jueves',
    4: 'Viernes',
    5: 'Sábado',
    6: 'Domingo'
}


def obtener_rango_horario(hora):
    hora_del_dia = hora #hora.hour
    for rango, (inicio, fin) in rangos_horarios.items():
        if inicio <= hora_del_dia < fin:
            return rango

# Función para determinar el rango estacional
def obtener_rango_estacional(fecha):
    mes_dia = (fecha.month, fecha.day)
    for estacion, (inicio, fin) in rangos_estacionales.items():
        if inicio <= mes_dia <= fin:
            return estacion

In [5]:
df_transformacion['RangoHorario'] = df_transformacion['hora'].apply(obtener_rango_horario)
onehot_encodrh = pd.get_dummies(df_transformacion['RangoHorario'], prefix='t')
df_transformacion = pd.concat([df_transformacion, onehot_encodrh], axis=1)
df_transformacion['Estacion'] = df_transformacion['fecha'].apply(obtener_rango_estacional)
df_transformacion.loc[df_transformacion['Estacion'] == 'Invierno2', 'Estacion'] = '1.Invierno'
onehot_encodEs = pd.get_dummies(df_transformacion['Estacion'], prefix='e')
df_transformacion = pd.concat([df_transformacion, onehot_encodEs], axis=1)

In [6]:
df_transformacion['id_DS'] = df_transformacion['fecha'].dt.dayofweek
df_transformacion['Dia_Semana'] = df_transformacion['fecha'].dt.dayofweek.map(dias_semana)
df_transformacion['NFS'] = df_transformacion['fecha'].dt.dayofweek.isin([0, 1, 2, 3, 4])
df_transformacion['FDS'] = df_transformacion['fecha'].dt.dayofweek.isin([5, 6])

In [7]:
df_transformacion[['id_DS','Dia_Semana','NFS','FDS']]

,id_DS,Dia_Semana,NFS,FDS
0,3,Jueves,True,False
1,3,Jueves,True,False
2,3,Jueves,True,False
3,3,Jueves,True,False
4,3,Jueves,True,False
...,...,...,...,...
355865,4,Viernes,True,False
355866,4,Viernes,True,False
355867,4,Viernes,True,False
355868,4,Viernes,True,False


In [8]:
cArtefCocin = ['Dish washer','Microwave','Kettle' ]
cArtefLavan = ['Washing Machine', 'Dryer']
cArtefConti = ['Water heater', 'Refrigerator']
cArtefEspac = ['Lighting']
cArtefEntre = ['TV']

df_transformacion['Cocina'] = df_transformacion[cArtefCocin].sum(axis=1)
df_transformacion['Lavanderia'] = df_transformacion[cArtefLavan].sum(axis=1)
df_transformacion['Continuo'] = df_transformacion[cArtefConti].sum(axis=1)
df_transformacion['EspacioComun'] = df_transformacion[cArtefEspac].sum(axis=1)
df_transformacion['Entretenimiento'] = df_transformacion[cArtefEntre].sum(axis=1)

df_transformacion

,fecha,Total Consumo,hora,Consumidor,Dish washer,Washing Machine,Dryer,Water heater,TV,Microwave,...,e_4.Otoño,id_DS,Dia_Semana,NFS,FDS,Cocina,Lavanderia,Continuo,EspacioComun,Entretenimiento
0,2013-08-01,0.140000,0,Consumer1,0.000000,0.000000,0.0,0.000,0.000000,0.0,...,False,3,Jueves,True,False,0.000000,0.000000,0.140,0.000000,0.000000
1,2013-08-01,0.280000,0,Consumer10,0.000000,0.000000,0.0,0.000,0.000000,0.0,...,False,3,Jueves,True,False,0.000000,0.000000,0.280,0.000000,0.000000
2,2013-08-01,1.178000,0,Consumer11,0.000000,0.000000,0.0,1.178,0.000000,0.0,...,False,3,Jueves,True,False,0.000000,0.000000,1.178,0.000000,0.000000
3,2013-08-01,1.005000,0,Consumer12,0.000000,0.000000,0.0,1.005,0.000000,0.0,...,False,3,Jueves,True,False,0.000000,0.000000,1.005,0.000000,0.000000
4,2013-08-01,2.522000,0,Consumer13,0.000000,0.000000,0.0,2.382,0.000000,0.0,...,False,3,Jueves,True,False,0.000000,0.000000,2.522,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355865,2014-08-01,0.140000,23,Consumer47,0.000000,0.000000,0.0,0.000,0.000000,0.0,...,False,4,Viernes,True,False,0.000000,0.000000,0.140,0.000000,0.000000
355866,2014-08-01,1.251331,23,Consumer49,1.251331,0.000000,0.0,0.000,0.000000,0.0,...,False,4,Viernes,True,False,1.251331,0.000000,0.000,0.000000,0.000000
355867,2014-08-01,1.882762,23,Consumer5,0.000000,1.882762,0.0,0.000,0.000000,0.0,...,False,4,Viernes,True,False,0.000000,1.882762,0.000,0.000000,0.000000
355868,2014-08-01,1.252174,23,Consumer50,0.000000,0.000000,0.0,0.659,0.093508,0.0,...,False,4,Viernes,True,False,0.000000,0.000000,0.659,0.499667,0.093508


In [12]:
df_transformacion.dtypes

fecha              datetime64[ns]
Total Consumo             float64
hora                        int64
Consumidor                 object
Dish washer               float64
Washing Machine           float64
Dryer                     float64
Water heater              float64
TV                        float64
Microwave                 float64
Kettle                    float64
Lighting                  float64
Refrigerator              float64
Temp                        int64
Producer                  float64
Is_producer                  bool
RangoHorario               object
t_1.Madrugada                bool
t_2.Mañana                   bool
t_3.Tarde                    bool
t_4.Noche                    bool
Estacion                   object
e_1.Invierno                 bool
e_2.Primavera                bool
e_3.Verano                   bool
e_4.Otoño                    bool
id_DS                       int32
Dia_Semana                 object
NFS                          bool
FDS           

In [13]:
from google.colab import files
df_transformacion.to_csv('03_transformacion.csv', header=True, index=False)
files.download('03_transformacion.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>